In [32]:
import sqlite3
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas_profiling import ProfileReport
import sklearn as sk
import dtale

In [2]:
df = pd.read_csv("donors.csv", low_memory=False).drop(columns="Unnamed: 0")

In [52]:
sample_df = df.iloc[:1000, :].copy()

# EDA

In [55]:
# The following variables indicate the number of
# known times the donor has responded to other
# types of mail order offers.

mailOrderOffers = sample_df.loc[:, "MBCRAFT":"PUBOPP"].columns.to_list()

In [ ]:
dtale.show(sample_df[mailOrderOffers])

In [46]:
pd.DataFrame([sample_df[mailOrderOffers].sum(1),sample_df.HIT]).T

,Unnamed 0,HIT
0,0.0,0.0
1,11.0,16.0
2,2.0,2.0
3,1.0,2.0
4,21.0,60.0
...,...,...
995,0.0,5.0
996,0.0,0.0
997,2.0,7.0
998,0.0,0.0


In [60]:
for col in mailOrderOffers:
    sample_df[col] = sample_df[col].fillna(0).astype(np.int16)

In [69]:
dtale.show(df.MAILCODE)

http://FSX-UBUNTU:40000/dtale/main/4
It looks like this data may have already been loaded to D-Tale based on shape and column names. Here is URL of the data that seems to match it:

None

If you still want to load this data please use the following command:

dtale.show(df, ignore_duplicate=True)
